## Introduction

Hello, my readers, I'm Parthiv. Quick note: **This file will have a lot of playing around and searching for trends!** The `dist/` directory will have a lot of the final graphs and analysis, so make sure to explore that after. I'll try to explain what I'm doing as we go on, but I apologize if anything's unclear.

### Table of Contents

- [EDA](/EDA)

In [3]:
# Importing necessary libraries
# pip install pandas numpy matplotlib seaborn plotly scienceplots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


### Config for Matplotlib, Seaborn, Plotly, LaTeX

Style config. Prioritizing aesthetics but also should be scientifically appropriate.

In [4]:
# Set global figure size
plt.rcParams['figure.figsize'] = [10, 6]

# Set font sizes
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

plt.rcParams['figure.autolayout'] = True

sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

import plotly.io as pio
pio.templates.default = "plotly_white"

plt.rcParams['text.usetex'] = True


In [5]:
# Color palettes

custom_palette = sns.color_palette('Set2')
sns.set_palette(custom_palette)

from cycler import cycler

# Define a custom color cycle
colors = ['#4E79A7', '#F28E2B', '#E15759', '#76B7B2', '#59A14F']
plt.rcParams['axes.prop_cycle'] = cycler(color=colors)


In [6]:
# Fonts
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

## Pre-processing
Load data into dataframes, perform initial inspection, handle missing values & incorrect data types

In [7]:
titles = pd.read_csv('titles.csv')
credits = pd.read_csv('credits.csv')
print("Titles Dataset:")
display(titles.head())

print("Credits Dataset:")
display(credits.head())


Titles Dataset:


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


Credits Dataset:


,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [8]:
print(f"Titles dataset shape: {titles.shape}")
print(f"Credits dataset shape: {credits.shape}")


Titles dataset shape: (5849, 15)
Credits dataset shape: (77801, 5)


In [9]:
print("Titles Dataset Info:")
print(titles.info())

print("\nCredits Dataset Info:")
print(credits.info())

Titles Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5849 entries, 0 to 5848
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5849 non-null   object 
 1   title                 5848 non-null   object 
 2   type                  5849 non-null   object 
 3   description           5831 non-null   object 
 4   release_year          5849 non-null   int64  
 5   age_certification     3230 non-null   object 
 6   runtime               5849 non-null   int64  
 7   genres                5849 non-null   object 
 8   production_countries  5849 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5446 non-null   object 
 11  imdb_score            5367 non-null   float64
 12  imdb_votes            5351 non-null   float64
 13  tmdb_popularity       5758 non-null   float64
 14  tmdb_score            5538 non-null   float64
dtype

In [10]:
print("Missing Values in Titles Dataset:")
print(titles.isnull().sum())

print("\nMissing Values in Credits Dataset:")
print(credits.isnull().sum())

print("Percentage of Missing Values in Titles Dataset:")
print((titles.isnull().sum() / len(titles)) * 100)

print("\nPercentage of Missing Values in Credits Dataset:")
print((credits.isnull().sum() / len(credits)) * 100)


Missing Values in Titles Dataset:
id                         0
title                      1
type                       0
description               18
release_year               0
age_certification       2619
runtime                    0
genres                     0
production_countries       0
seasons                 3743
imdb_id                  403
imdb_score               482
imdb_votes               498
tmdb_popularity           91
tmdb_score               311
dtype: int64

Missing Values in Credits Dataset:
person_id       0
id              0
name            0
character    9772
role            0
dtype: int64
Percentage of Missing Values in Titles Dataset:
id                       0.000000
title                    0.017097
type                     0.000000
description              0.307745
release_year             0.000000
age_certification       44.776885
runtime                  0.000000
genres                   0.000000
production_countries     0.000000
seasons                 6

In [11]:
titles = titles.dropna(subset=['title'])
titles['description'] = titles['description'].fillna('No description available.')
titles['age_certification'] = titles['age_certification'].fillna('Not Rated')
titles['seasons'] = titles['seasons'].fillna(0).astype(int)
credits['character'] = credits['character'].fillna('Unknown Character')

In [12]:
titles['runtime'] = titles['runtime'].astype(int)
titles['release_year'] = titles['release_year'].astype(int)
titles = titles.drop_duplicates()
credits = credits.drop_duplicates()
titles['type'] = titles['type'].str.strip().str.upper()

## EDA
EDA - Exploratory data analysis


We will try to explore four things here:
- Shift Towards Diversity and Social Issues
- Global Expansion Reflected in Production Countries
- Simplification of Content Descriptions Over Time
- Diversity in the Acting Crew Over Time

We will start with the shift towards diversity and social issues.


### Shift towards Diversity and Social Issues
- Demonstrate an increase in Netflix content focusing on diversity and social issues over time.
- NLP (Natural Language Processing) to analyze descriptions and titles for relevant keywords, sentiment, and topics.
- Visualize trends to support the claim.

In [17]:
# Relevant keywords
diversity_keywords = [
    'diversity', 'inclusion', 'equality', 'race', 'ethnicity', 'gender', 'identity', 'lgbtq', 'minority', 'empowerment',
    'mental health', 'climate change', 'environment', 'justice', 'poverty', 'discrimination', 'immigration', 'violence',
    'abuse', 'human rights', 'activism', 'social justice', 'culture', 'civil rights', 'intersectionality', 'accessibility',
    'sustainability', 'refugee', 'gender equality', 'women', 'indigenous', 'religion', 'class', 'ageism', 'ableism'
]

In [35]:
import re
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Parthiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Parthiv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Parthiv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We will now do a little preprocessing for this analysis

In [37]:
titles['description'] = titles['description'].astype(str).str.lower() # Convert to lowercase
# Use a regex to remove every character except the alphabet
titles['description'] = titles['description'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

titles['tokens'] = titles['description'].apply(word_tokenize)
stop_words = set(stopwords.words('english'))

# remove stopwords like of, on, an, in, to
titles['tokens'] = titles['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

print(titles.head())

         id                                title   type  \
0  ts300399  Five Came Back: The Reference Films   SHOW   
1   tm84618                          Taxi Driver  MOVIE   
2  tm154986                          Deliverance  MOVIE   
3  tm127384      Monty Python and the Holy Grail  MOVIE   
4  tm120801                      The Dirty Dozen  MOVIE   

                                         description  release_year  \
0  this collection includes  world war iiera prop...          1945   
1  a mentally unstable vietnam war veteran works ...          1976   
2  intent on seeing the cahulawassee river before...          1972   
3  king arthur accompanied by his squire recruits...          1975   
4   american military prisoners in world war ii a...          1967   

  age_certification  runtime                                       genres  \
0             TV-MA       51                            ['documentation']   
1                 R      114                           ['drama', 'crim